In [1]:
from dataclasses import dataclass
from pathlib import Path

from aoc.decorators import timeit

data_file = Path("../Data/day8.txt").read_text()

EXAMPLE = """..........
..........
..........
....a.....
..........
.....a....
..........
..........
..........
.........."""

EXAMPLE2 = """..........
..........
..........
....a.....
........a.
.....a....
..........
..........
..........
.........."""

EXAMPLE3 = """..........
..........
..........
....a.....
........a.
.....a....
..........
......A...
..........
.........."""

EXAMPLE4 = """............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............"""


@dataclass(frozen=True, eq=True)
class Point:
    x: int
    y: int

    @staticmethod
    def is_collinear(point_a: "Point", point_b: "Point", point_c: "Point"):
        x1, y1 = point_b.x - point_a.x, point_b.y - point_a.y
        x2, y2 = point_c.x - point_a.x, point_c.y - point_a.y

        return abs(x1 * y2 - x2 * y1) < 1e-12


@dataclass
class RoofPlot:
    point: Point
    value: str


class RoofMap:
    plots: list[list[RoofPlot]]
    anti_nodes: set[Point]
    grouped_antennas: dict[str, list[RoofPlot]]

    def __init__(self, plots: list[list[RoofPlot]]) -> None:
        self.plots = plots
        self.grouped_antennas = RoofMap.__get_antennas(plots)
        self.anti_nodes = set()

    def count_anti_nodes(self):
        # TODO: maybe make sure the anti_nodes are not out of bound
        anti_nodes_count = 0
        for row in self.plots:
            for plot in row:
                if plot.point in self.anti_nodes:
                    anti_nodes_count += 1

        return anti_nodes_count

    def place_anti_node(self, point: Point):
        self.anti_nodes.add(point)

    def print_antennas(self):
        for row in self.plots:
            print("".join(map(lambda x: x.value, row)))

    def print_anti_nodes(self):
        for row in self.plots:
            print(
                "".join(
                    map(lambda x: "#" if x.point in self.anti_nodes else x.value, row)
                )
            )

    @staticmethod
    def from_input(input: str):
        def map_line(enumerated_line: tuple[int, str]):
            x, line = enumerated_line

            return list(
                map(
                    lambda enumerated_value: RoofPlot(
                        point=Point(x=x, y=enumerated_value[0]),
                        value=enumerated_value[1],
                    ),
                    enumerate(line),
                )
            )

        return RoofMap(plots=list(map(map_line, enumerate(input.splitlines()))))

    @staticmethod
    def __get_antennas(plots: list[list[RoofPlot]]):
        antennas: dict[str, list[RoofPlot]] = {}
        for row in plots:
            for plot in row:
                if plot.value == ".":
                    continue

                antennas[plot.value] = antennas.get(plot.value, []) + [plot]

        return antennas


def prepare(input: str):
    return RoofMap.from_input(input=input)

In [2]:
def make_pairs(antennas: list[RoofPlot]) -> list[tuple[RoofPlot, RoofPlot]]:
    pairs: list[tuple[RoofPlot, RoofPlot]] = []
    if len(antennas) < 2:
        return []

    for i, antenna_a in enumerate(antennas[:-1]):
        for antenna_b in antennas[i + 1 :]:
            pairs.append((antenna_a, antenna_b))

    return pairs


def get_anti_nodes(antennas: list[RoofPlot]):
    pairs = make_pairs(antennas)
    anti_nodes: list[Point] = []
    for antenna_a, antenna_b in pairs:
        x_difference = abs(antenna_a.point.x - antenna_b.point.x)
        y_difference = abs(antenna_a.point.y - antenna_b.point.y)

        potential_anti_nodes = [
            Point(
                x=antenna_a.point.x - x_difference,
                y=antenna_a.point.y - y_difference,
            ),
            Point(
                x=antenna_a.point.x - x_difference,
                y=antenna_a.point.y + y_difference,
            ),
            Point(
                x=antenna_b.point.x + x_difference,
                y=antenna_b.point.y + y_difference,
            ),
            Point(
                x=antenna_b.point.x + x_difference,
                y=antenna_b.point.y - y_difference,
            ),
        ]
        for node in potential_anti_nodes:
            if Point.is_collinear(antenna_a.point, antenna_b.point, node):
                anti_nodes.append(node)

    return anti_nodes


@timeit
def part1(input: str, *, debug=False):
    roof_map = prepare(input)
    for antennas in roof_map.grouped_antennas.values():
        anti_nodes = get_anti_nodes(antennas)
        for anti_node in anti_nodes:
            roof_map.place_anti_node(anti_node)

    if debug:
        roof_map.print_anti_nodes()

    return roof_map.count_anti_nodes()


example_result = part1(EXAMPLE)

assert (
    example_result == 2
), f"Expected example result to be 2, but got {example_result} instead"

example_result2 = part1(EXAMPLE2)

assert (
    example_result2 == 4
), f"Expected example 2 result to be 4, but got {example_result2} instead"

example_result3 = part1(EXAMPLE3)

assert (
    example_result3 == 4
), f"Expected example 3 result to be 4, but got {example_result3} instead"

example_result4 = part1(EXAMPLE4, debug=False)

assert (
    example_result4 == 14
), f"Expected example 4 result to be 14, but got {example_result4} instead"

result = part1(data_file, debug=False)

assert result < 330, f"Expected result to be less than 330, but got {result} instead"
assert result < 307, f"Expected result to be less than 307, but got {result} instead"
assert result > 285, f"Expected result to be greater than 285, but got {result} instead"
assert result == 303, f"Expected result to be 303, but got {result} instead"

print("result is", result)

def part1(input, debug): took: 0.0001 sec
def part1(input, debug): took: 0.0004 sec
def part1(input, debug): took: 0.0004 sec
def part1(input, debug): took: 0.0007 sec
def part1(input, debug): took: 0.0108 sec
result is 303


In [3]:
@timeit
def part2(input: str):
    prepare(input)
    return 0


example_result = part2(EXAMPLE)

print("example result is", example_result)

assert example_result == 0

result = part2(data_file)

print("result is", result)

assert result == 0

def part2(input): took: 0.0001 sec
example result is 0
def part2(input): took: 0.0014 sec
result is 0
